# Lab 3 - Machine Learning 

## Marketing Digital Prevendo Número de Usuários Convertidos

**Descrição projeto**

Você foi contratado como Cientista de Dados por uma empresa que comercializa produtos digitais. A empresa trabalha diversas estratégias de Marketing Digital e gostaria de ter um modelo de Machine Learning capaz de prever quantos usuários serão convertidos (ou seja,quantas pessoas comprarão os produtos da empresa) após cada campanha. 

Conseguindo fazer a previsão, a empresa pode ter uma ideia mais clara de quanto deve investir em cada campanha e
o retorno esperado. Isso ajudará também no planejamento da empresa para comercialização e entrega do seu produto digital, além do uso de ferramentas e mídias sociais.

Dados históricos de campanhas passadas estão disponíveis e seu trabalho como Cientista de Dados é construir um modelo que, ao receber novos dados, seja capaz de prever o número de usuários convertidos em uma campanha de Marketing Digital. Além disso, o Gestor de Marketing precisa saber qual seria o aumento no número de usuários convertidos se aumentar em 1
unidade o valor gasto em uma campanha. 

Entretanto, os dados têm problemas (exatamente o que você encontrará no dia a dia) e você deve detectar esses problemas, decidir a melhor estratégia para resolvê-los e então criar seu modelo. Pode ser necessário criar diferentes versões do modelo até chegar ao modelo ideal. 

Quando chegar à versão ideal do modelo, você deve fornecer uma interpretação completa de como o modelo gera o resultado final para que os gestores tenham mais confiança no uso do modelo.

Por fim, você deve fornecer uma forma de fazer o deploy do modelo e usá-lo imediatamente com novos dados.

**Obejtivos**

**Problema 2 - Pergunta de Negócio do Lab 3**

Um Lead será convertido? Sim ou Não? Qual a probabilidade? (Classificação)

In [1]:
# Importação das bibliotecas 

# Bibliotecas sistema
import re
import random
import unicodedata
import itertools

# Biblioteca para manipulação de arquivos
import pandas as pd
import numpy as np 


# Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# Carregar as versões das bibliotecas
import watermark

# Versões das bibliotecas
%reload_ext watermark
%watermark -a "Versões das bibliotecas" --iversions

# Configuração para os gráficos largura e layout dos graficos
sns.set_palette("Accent")
sns.set(style="whitegrid", color_codes=True, font_scale=1.8)
color = sns.color_palette()

# Warnings retirar alertas 
import warnings
warnings.filterwarnings("ignore")

# Versão do python
from platform import python_version
print('Versão python neste Jupyter Notebook:', python_version())

Author: Versões das bibliotecas

numpy     : 1.23.5
seaborn   : 0.12.2
re        : 2.2.1
matplotlib: 3.7.2
pandas    : 2.0.3
watermark : 2.4.3

Versão python neste Jupyter Notebook: 3.9.13


## Base dados

In [3]:
# Base dados
data = pd.read_csv("dataset.csv")
data

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

In [ ]:
# Visualizando os 5 primeiros dados
data.head()

In [ ]:
# Visualizando os 5 últimos dados
data.tail()

In [ ]:
# Info dados
data.info()

In [ ]:
# Tipo dados
data.dtypes

In [ ]:
# Total de colunas e linhas 
print("Números de linhas: {}" .format(data.shape[0]))
print("Números de colunas: {}" .format(data.shape[1]))

In [ ]:
# Exibindo valores ausentes e valores únicos
print("\nMissing values\n", data.isnull().sum().values.sum())
print("\nUnique values\n",data.nunique())

In [ ]:
# Verificando dados ausentes
data.isna().sum()

In [ ]:
# Dados duplicados
data.duplicated().sum()

## Análise exploratória de dados

In [ ]:
# Gráfico de barras
plt.figure(figsize=(10, 8))
sns.countplot(data=data, x='converteu', hue="converteu")

# Adicione rótulos aos eixos e um título
plt.xlabel('Converteu')
plt.ylabel('Contagem')
plt.title("Distribuição da Variável 'Converteu'")

# Exiba o gráfico
plt.show()

In [ ]:
# Boxplot
plt.figure(figsize=(10, 8))
sns.boxplot(data = data, x="converteu", y="numero_cliques", hue="converteu")
# Adicione rótulos aos eixos e um título
plt.xlabel('Converteu')
plt.ylabel('Número de Cliques')
plt.title("Boxplot - Número de Cliques por Conversão'")

# Exiba o gráfico
plt.show()

In [ ]:
# Gráfico de barras

plt.figure(figsize=(10, 6))
sns.countplot(data=data, x='faixa_etaria')

# Adicione rótulos aos eixos e um título
plt.xlabel('Faixa Etária')
plt.ylabel('Contagem')
plt.title('Distribuição de Faixa Etária')

# Rotacione os rótulos do eixo x para melhorar a legibilidade (opcional)
plt.xticks(rotation=45)

# Exiba o gráfico
plt.show()

In [ ]:
# Gráfico de dispersão
plt.figure(figsize=(10, 8))  
sns.scatterplot(data=data, x='numero_acessos', y='numero_cliques', hue="converteu")

# Adicione rótulos aos eixos e um título
plt.xlabel('Número de Acessos')
plt.ylabel('Número de Cliques')
plt.title('Relação entre Número de Acessos e Número de Cliques')

# Exiba o gráfico
plt.show()

In [ ]:
# Use o método groupby() para agrupar os dados pela coluna 'cidade' e calcule a média usando o método mean()
dados_sumarizados = data.groupby('cidade')['numero_acessos'].mean().reset_index()

# O resultado será um novo DataFrame com a média do número de acessos por cidade
print(dados_sumarizados)

In [ ]:
# Gráfico de barras
plt.figure(figsize=(10, 8))  
sns.barplot(x="cidade", y="numero_acessos", data=dados_sumarizados)
# Adicione rótulos aos eixos e um título
plt.xlabel('Cidade')
plt.ylabel('Média do Número de Acessos')
plt.title('Gráfico de Barras - Média do Número de Acessos por Cidade')

# Exiba o gráfico
plt.show()

## Features modelo

In [ ]:
# Divida os dados em recursos (X) e variável alvo (y)

# Recursos
x = data.drop('converteu', axis=1)  

# Variável alvo
y = data['converteu']

In [ ]:
# Visualizando features (x)
x.shape

In [ ]:
# Visualizando features (y)
y.shape

## Pré-Processamento e Encoding de Variáveis Categóricas

Pré-processamento e codificação (encoding) de variáveis categóricas são etapas importantes na preparação de dados para modelos de machine learning. Abaixo, descreverei essas etapas e fornecerei um exemplo de como realizar essas operações usando Python e a biblioteca pandas.

## Pré-Processamento de Variáveis Categóricas

**Identificação de Variáveis Categóricas:** Primeiro, identifique as colunas do seu conjunto de dados que contêm variáveis categóricas. Essas são colunas que contêm informações não numéricas, como categorias, nomes, cores, etc.


**Tratamento de Valores Ausentes:** Verifique se há valores ausentes nas variáveis categóricas e decida como tratá-los (por exemplo, preenchendo com um valor padrão ou eliminando as linhas com valores ausentes).


**Codificação de Variáveis Categóricas:** Converta as variáveis categóricas em um formato numérico que os modelos de machine learning possam entender. Existem várias técnicas para fazer isso.


**Codificação de Variáveis Categóricas**


**Label Encoding:** Esta técnica é usada quando uma variável categórica tem duas categorias únicas. Por exemplo, "sim" e "não" podem ser codificadas como 1 e 0, respectivamente.


**One-Hot Encoding:** É usada quando uma variável categórica tem mais de duas categorias únicas. Cria colunas binárias (0 ou 1) para cada categoria. O pandas possui uma função get_dummies() que facilita essa codificação.

In [ ]:
## Aplicando label encoding à variável alvo

# Importando biblioteca 
from sklearn.preprocessing import LabelEncoder

# Codifique as variáveis categóricas, se necessário
le = LabelEncoder()
X_encoded = x.apply(lambda col: le.fit_transform(col) if col.dtype == 'object' else col)
le

In [ ]:
# Visualizando dados
X_encoded.shape

In [ ]:
# Aplique o Label Encoding às variáveis categóricas
data['cidade'] = le.fit_transform(data['cidade'])
data['cor_da_pele'] = le.fit_transform(data['cor_da_pele'])
data['navegador_web'] = le.fit_transform(data['navegador_web'])

# Visualizando 
data

# Feature Engineering

Praticamente todos os algoritmos de Aprendizado de Máquina possuem entradas e saídas. As entradas são formadas por colunas de dados estruturados, onde cada coluna recebe o nome de feature, também conhecido como variáveis independentes ou atributos. Essas features podem ser palavras, pedaços de informação de uma imagem, etc. Os modelos de aprendizado de máquina utilizam esses recursos para classificar as informações.
Por exemplo, sedentarismo e fator hereditário são variáveis independentes para quando se quer prever se alguém vai ter câncer ou não

As saídas, por sua vez, são chamadas de variáveis dependentes ou classe, e essa é a variável que estamos tentando prever. O nosso resultado pode ser 0 e 1 correspondendo a 'Não' e 'Sim' respectivamente, que responde a uma pergunta como: "Fulano é bom pagador?" ou a probabilidade de alguém comprar um produto ou não.

In [ ]:
from sklearn.preprocessing import LabelEncoder

for i in data.columns:
    if data[i].dtype==np.number:
        continue
    data[i]= LabelEncoder().fit_transform(data[i])
    
data.head(4)

In [ ]:
# Divida os dados em recursos (X) e variável alvo (y)

# Recursos
data_train_x = data.drop('converteu', axis=1)  

# Variável alvo
data_train_y = data['converteu']

In [ ]:
# Visualizando features (x)
x.shape

In [ ]:
# Visualizando features (y)
y.shape

In [ ]:
# Salvando dataset limpo
data.to_csv("datataset_final.csv")

## Treino teste

- Dividir o conjunto de dados em treinamento e teste

In [ ]:
# Importando biblioteca
from sklearn.model_selection import train_test_split

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data_train_x, 
                                                    data_train_y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
# Visualizando dados treino
X_train.shape

In [ ]:
# Visualizando dados teste
y_train.shape

## Machine learning - Modelagem Preditiva

**Modelo 01 - Regressão logística**

In [ ]:
%%time

# Importando biblioteca
from sklearn.linear_model import LogisticRegression

# Criando modelo regressão logistica
modelo_regressao_logistica = LogisticRegression()

# Treinamento modelo
modelo_regressao_logistica_fit = modelo_regressao_logistica.fit(X_train, y_train)

# Visualizando modelo
modelo_regressao_logistica

In [ ]:
# Score modelo
modelo_score = modelo_regressao_logistica.score(X_train, y_train)
print("Model - Regressão logistica: %.2f" % (modelo_score * 100))

In [ ]:
# Previsão modelo
modelo_regressao_logistica_pred = modelo_regressao_logistica.predict(X_test)
modelo_regressao_logistica_pred

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_regressao_logistica = accuracy_score(y_test, modelo_regressao_logistica_pred)
print("Accuracy - Regressão logistica: %.2f" % (accuracy_regressao_logistica * 100))

In [ ]:
from sklearn.metrics import confusion_matrix

matrix_confusion = confusion_matrix(y_test, modelo_regressao_logistica_pred)

print('Confusion matrix - Regressão logistica \n\n', matrix_confusion)
print('\nTrue Positives(TP) = ', matrix_confusion[0,0])
print('\nTrue Negatives(TN) = ', matrix_confusion[1,1])
print('\nFalse Positives(FP) = ', matrix_confusion[0,1])
print('\nFalse Negatives(FN) = ', matrix_confusion[1,0])

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 5))

ax = plt.subplot()
sns.heatmap(matrix_confusion, annot=True, ax = ax, fmt = ".1f", cmap="Paired"); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(["Acesso", "Não acesso"]); ax.yaxis.set_ticklabels(["Acesso", "Não acesso"]);

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

roc = modelo_regressao_logistica.predict_proba(X_test)[:,1]
tfp, tvp, limite = roc_curve(y_test, roc)
print('roc_auc', roc_auc_score(y_test, roc))

plt.subplots(1, figsize=(10,8))
plt.title('Curva ROC - Regressão logistica')
plt.plot(tfp,tvp)
plt.xlabel('Especifidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], ls="--", c = 'red')
plt.plot([0, 0], [1, 0], ls="--", c = 'green'), plt.plot([1, 1], ls="--", c = 'green')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

classification = classification_report(y_test, modelo_regressao_logistica_pred)
print("Modelo 01 - Regressão logistica")
print()
print(classification)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

precision = precision_score(y_test, modelo_regressao_logistica_pred)
Recall = recall_score(y_test, modelo_regressao_logistica_pred)
Accuracy = accuracy_score(y_test, modelo_regressao_logistica_pred)
F1_Score = f1_score(y_test, modelo_regressao_logistica_pred)

precisao = pd.DataFrame({
    
    "Metricas" : ["precision",
                 "Recall", 
                  "Accuracy", 
                  "F1_Score"],
    
    "Resultado": [precision,
                Recall, 
                Accuracy, 
                F1_Score]})

precisao.sort_values(by = "Resultado", ascending = False)

## Modelo 02 - Decision Tree

In [ ]:
%%time

# Importando biblioteca
from sklearn.tree import DecisionTreeClassifier

# Criando modelo 
modelo_arvore_cla_1 = DecisionTreeClassifier(max_depth=4, random_state=0)

# Treinamento modelo
modelo_arvore_cla_fit = modelo_arvore_cla_1.fit(X_train, y_train)

# Visualizando modelo
modelo_arvore_cla_1

In [ ]:
# Score modelo
modelo_arvore_scor = modelo_arvore_cla_1.score(X_train, y_train)
print("Modelo - Decision Tree Classifier: %.2f" % (modelo_arvore_scor * 100))

In [ ]:
# Previsão do modelo
modelo_arvore_pred = modelo_arvore_cla_1.predict(X_test)
modelo_arvore_pred

In [ ]:
# Accuracy
acuracia_decision_tree = accuracy_score(y_test, modelo_arvore_pred)
print("Acuracia - Decision Tree: %.2f" % (acuracia_decision_tree * 100))

In [ ]:
roc = modelo_arvore_cla_1.predict_proba(X_test)[:,1]
tfp, tvp, limite = roc_curve(y_test, roc)
print('roc_auc', roc_auc_score(y_test, roc))

plt.subplots(1, figsize=(5,5))
plt.title('Curva ROC - Arvore')
plt.plot(tfp,tvp)
plt.xlabel('Especifidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], ls="--", c = 'red')
plt.plot([0, 0], [1, 0], ls="--", c = 'green'), plt.plot([1, 1], ls="--", c = 'green')
plt.show()

In [ ]:
# confusion matrix 
matrix_confusion2 = confusion_matrix(y_test, modelo_arvore_pred)

print('Confusion matrix - Decision Tree \n\n', matrix_confusion2)
print('\nTrue Positives(TP) = ', matrix_confusion2[0,0])
print('\nTrue Negatives(TN) = ', matrix_confusion2[1,1])
print('\nFalse Positives(FP) = ', matrix_confusion2[0,1])
print('\nFalse Negatives(FN) = ', matrix_confusion2[1,0])

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 5))

ax = plt.subplot()
sns.heatmap(matrix_confusion2, annot=True, ax = ax, fmt = ".1f", cmap="Paired"); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(["Acesso", "Não acesso"]); ax.yaxis.set_ticklabels(["Acesso", "Não acesso"]);

In [ ]:
class_report = classification_report(y_test, modelo_arvore_pred)
print("Modelo - Decision Tree")
print("\n")
print(class_report)

In [ ]:
precision = precision_score(y_test, modelo_arvore_pred)
Recall = recall_score(y_test, modelo_arvore_pred)
Accuracy = accuracy_score(y_test, modelo_arvore_pred)
F1_Score = f1_score(y_test, modelo_arvore_pred)

precisao = pd.DataFrame({
    
    "Metricas" : ["precision",
                 "Recall", 
                  "Accuracy", 
                  "F1_Score"],
    
    "Resultado": [precision,
                Recall, 
                Accuracy, 
                F1_Score]})

precisao.sort_values(by = "Resultado", ascending = False)

## Modelo 03 - Naive bayes

In [ ]:
%%time

# Importando biblioteca
from sklearn.naive_bayes import GaussianNB

# Criando modelo
model_naive_bayes = GaussianNB()

# Treinamento modelo
model_naive_bayes_fit = model_naive_bayes.fit(X_train, y_train)

# Visualizando modelo
model_naive_bayes

In [ ]:
# Score modelo
model_naive_bayes_score = model_naive_bayes.score(X_train, y_train)
print("Modelo - Naive Bayes: %.2f" % (model_naive_bayes_score * 100))

In [ ]:
# Previsão do modelo - Naive bayes
model_naive_bayes_pred_predict = model_naive_bayes.predict(X_test)
model_naive_bayes_pred_predict

In [ ]:
# Previsão com função probabiliestico do modelo - Naive bayes
model_naive_bayes_pred = model_naive_bayes.predict_proba(X_test)
model_naive_bayes_pred

In [ ]:
# Accuracy model
accuracy = accuracy_score(y_test, model_naive_bayes_pred_predict)
print("Accuracy Naive bayes: %.2f" % (accuracy * 100))

In [ ]:
# Matrix confusion
matrix_confusion3 = confusion_matrix(y_test, model_naive_bayes_pred_predict)

print('Confusion matrix - Naive bayes \n\n', matrix_confusion3)
print('\nTrue Positives(TP) = ', matrix_confusion3[0,0])
print('\nTrue Negatives(TN) = ', matrix_confusion3[1,1])
print('\nFalse Positives(FP) = ', matrix_confusion3[0,1])
print('\nFalse Negatives(FN) = ', matrix_confusion3[1,0])

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 5))

ax = plt.subplot()
sns.heatmap(matrix_confusion3, annot=True, ax = ax, fmt = ".1f", cmap="Paired"); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(["Acesso", "Não acesso"]); 
ax.yaxis.set_ticklabels(["Acesso", "Não acesso"]);

In [ ]:
roc = model_naive_bayes.predict_proba(X_test)[:,1]
tfp, tvp, limite = roc_curve(y_test, roc)
print('roc_auc', roc_auc_score(y_test, roc))

plt.subplots(1, figsize=(5,5))
plt.title('Curva ROC - Naive bayes')
plt.plot(tfp,tvp)
plt.xlabel('Especifidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], ls="--", c = 'red')
plt.plot([0, 0], [1, 0], ls="--", c = 'green'), plt.plot([1, 1], ls="--", c = 'green')
plt.show()

In [ ]:
# Classification report modelo
class_report = classification_report(y_test, model_naive_bayes_pred_predict)
print("Modelo 03 - Naive Bayes")
print("\n")
print(class_report)

In [ ]:
# Métricas modelo
precision = precision_score(y_test, model_naive_bayes_pred_predict)
Recall = recall_score(y_test, model_naive_bayes_pred_predict)
Accuracy = accuracy_score(y_test, model_naive_bayes_pred_predict)
F1_Score = f1_score(y_test, model_naive_bayes_pred_predict)

precisao = pd.DataFrame({
    
    "Metricas" : ["precision",
                 "Recall", 
                  "Accuracy", 
                  "F1_Score"],
    
    "Resultado": [precision,
                Recall, 
                Accuracy, 
                F1_Score]})

precisao.sort_values(by = "Resultado", ascending = False)

## Modelo 04 - Random Forest

In [ ]:
%%time

# Importando modelo
from sklearn.ensemble import RandomForestClassifier

# Treinamento modelo max_depth - determinando total de árvore, random_state 0
model_random_forest = RandomForestClassifier(max_depth = 2, random_state = 0) 

# Dados de treino, teste de x, y
model_random_forest_fit = model_random_forest.fit(X_train, y_train)

# Visualizando modelo
model_random_forest

In [ ]:
# Valor da Accuracy do algoritmo 
model_random_forest_score = model_random_forest.score(X_train, y_train)
print("Score - Modelo random forest: %.2f" % (model_random_forest_score * 100))

In [ ]:
# Previsão do modelo
model_random_forest_regressor_pred = model_random_forest.predict(X_test)
model_random_forest_regressor_pred

In [ ]:
# Accuracy modelo
accuracy_random_forest = accuracy_score(y_test, model_random_forest_regressor_pred)
print("Accuracy - Random forest: %.2f" % (accuracy_random_forest * 100))

In [ ]:
# Confusion matrix
matrix_confusion4 = confusion_matrix(y_test, model_random_forest_regressor_pred)

print('Confusion matrix - Random forest \n\n', matrix_confusion4)
print('\nTrue Positives(TP) = ', matrix_confusion4[0,0])
print('\nTrue Negatives(TN) = ', matrix_confusion4[1,1])
print('\nFalse Positives(FP) = ', matrix_confusion4[0,1])
print('\nFalse Negatives(FN) = ', matrix_confusion4[1,0])

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 5))

ax = plt.subplot()
sns.heatmap(matrix_confusion4, annot=True, ax = ax, fmt = ".1f", cmap="Paired"); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(["Acesso", "Não acesso"]); 
ax.yaxis.set_ticklabels(["Acesso", "Não acesso"]);

In [ ]:
# Curva ROC
roc = model_random_forest.predict_proba(X_test)[:,1]
tfp, tvp, limite = roc_curve(y_test, roc)
print('roc_auc', roc_auc_score(y_test, roc))

plt.subplots(1, figsize=(5,5))
plt.title('Curva ROC')
plt.plot(tfp,tvp)
plt.xlabel('Especifidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], ls="--", c = 'red')
plt.plot([0, 0], [1, 0], ls="--", c = 'green'), plt.plot([1, 1], ls="--", c = 'green')
plt.show()

In [ ]:
# classification report
classification = classification_report(y_test, model_random_forest_regressor_pred)
print("Modelo 04 - Random forest")
print()
print(classification)

In [ ]:
# Métricas modelo modelo Random forest
precision = precision_score(y_test, model_random_forest_regressor_pred)
Recall = recall_score(y_test, model_random_forest_regressor_pred)
Accuracy = accuracy_score(y_test, model_random_forest_regressor_pred)
F1_Score = f1_score(y_test, model_random_forest_regressor_pred)

precisao = pd.DataFrame({
    
    "Metricas" : ["precision",
                 "Recall", 
                  "Accuracy", 
                  "F1_Score"],
    
    "Resultado": [precision,
                Recall, 
                Accuracy, 
                F1_Score]})

precisao.sort_values(by = "Resultado", ascending = False)

## Modelo 05 - XGBoost

In [ ]:
%%time

# Importando modelo
from xgboost import XGBClassifier

# Criando modelo
model_xgb = XGBClassifier()

# Treinamento modelo
model_xgb_fit = model_xgb.fit(X_train, y_train)

# Visualizando modelo
model_xgb

In [ ]:
# Score modelo
model_xgb_score = model_xgb.score(X_train, y_train)
print("Modelo - XGBoost: %.2f" % (model_xgb_score * 100))

In [ ]:
# Previsão do modelo - XGBoost

xgb_pred = model_xgb.predict(X_test)
xgb_pred

In [ ]:
# Accuracy modelo
accuracy_XGBoost = accuracy_score(y_test, xgb_pred)
print("Accuracy - XGBoost: %.2f" % (accuracy_XGBoost * 100))

In [ ]:
# Confusion matrix modelo
matrix_confusion5 = confusion_matrix(y_test, xgb_pred)

print('Confusion matrix - XGBoost \n\n', matrix_confusion5)
print('\nTrue Positives(TP) = ', matrix_confusion5[0,0])
print('\nTrue Negatives(TN) = ', matrix_confusion5[1,1])
print('\nFalse Positives(FP) = ', matrix_confusion5[0,1])
print('\nFalse Negatives(FN) = ', matrix_confusion5[1,0])

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 5))

ax = plt.subplot()
sns.heatmap(matrix_confusion5, annot=True, ax = ax, fmt = ".1f", cmap="Paired"); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(["Acesso", "Não acesso"]); 
ax.yaxis.set_ticklabels(["Acesso", "Não acesso"]);

In [ ]:
# Curva ROC
roc = model_xgb.predict_proba(X_test)[:,1]
tfp, tvp, limite = roc_curve(y_test, roc)
print('roc_auc', roc_auc_score(y_test, roc))

plt.subplots(1, figsize=(5,5))
plt.title('Curva ROC')
plt.plot(tfp,tvp)
plt.xlabel('Especifidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], ls="--", c = 'red')
plt.plot([0, 0], [1, 0], ls="--", c = 'green'), plt.plot([1, 1], ls="--", c = 'green')
plt.show()

In [ ]:
# Classification report
classification = classification_report(y_test, xgb_pred)
print("Modelo 05 - XGBoost")
print()
print(classification)

In [ ]:
# Métricas modelo
precision = precision_score(y_test, xgb_pred)
Recall = recall_score(y_test, xgb_pred)
Accuracy = accuracy_score(y_test, xgb_pred)
F1_Score = f1_score(y_test, xgb_pred)

precisao = pd.DataFrame({
    
    "Metricas" : ["precision",
                 "Recall", 
                  "Accuracy", 
                  "F1_Score"],
    
    "Resultado": [precision,
                Recall, 
                Accuracy, 
                F1_Score]})

precisao.sort_values(by = "Resultado", ascending = False)

## Modelo 06 - Gradient Boosting

In [ ]:
%%time

# Importando biblioteca
from sklearn.ensemble import GradientBoostingClassifier

# Criando modelo
model_gradient_boosting = GradientBoostingClassifier()

# Treinamento modelo
model_gradient_boosting_fit = model_gradient_boosting.fit(X_train, y_train)

# Visualizando modelo
model_gradient_boosting

In [ ]:
# Score modelo
model_gradient_boosting_score = model_gradient_boosting.score(X_train, y_train)
print("Modelo Score do Gradient Boosting: %.2f" % (model_gradient_boosting_score * 100))

In [ ]:
# Previsão do modelo - Gradient Boosting

model_gradient_boosting_pred = model_gradient_boosting.predict(X_test)
model_gradient_boosting_pred

In [ ]:
# Accuracy score modelo
accuracy_model_gradient_boosting = accuracy_score(y_test, model_gradient_boosting_pred)
print("Acurácia - Gradient boosting: %.2f" % (accuracy_model_gradient_boosting * 100))

In [ ]:
# Confusion matrix
matrix_confusion6 = confusion_matrix(y_test, model_gradient_boosting_pred)

print('Confusion matrix - Gradient Boosting \n\n', matrix_confusion6)
print('\nTrue Positives(TP) = ', matrix_confusion6[0,0])
print('\nTrue Negatives(TN) = ', matrix_confusion6[1,1])
print('\nFalse Positives(FP) = ', matrix_confusion6[0,1])
print('\nFalse Negatives(FN) = ', matrix_confusion6[1,0])

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 5))

ax = plt.subplot()
sns.heatmap(matrix_confusion6, annot=True, ax = ax, fmt = ".1f", cmap="Paired"); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(["Acesso", "Não acesso"]); 
ax.yaxis.set_ticklabels(["Acesso", "Não acesso"]);

In [ ]:
# Curva ROC
roc = model_gradient_boosting.predict_proba(X_test)[:,1]
tfp, tvp, limite = roc_curve(y_test, roc)
print('roc_auc', roc_auc_score(y_test, roc))

plt.subplots(1, figsize=(10,5))
plt.title('Curva ROC - Gradient boosting')
plt.plot(tfp,tvp)
plt.xlabel('Especifidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], ls="--", c = 'red')
plt.plot([0, 0], [1, 0], ls="--", c = 'green'), plt.plot([1, 1], ls="--", c = 'green')
plt.show()

In [ ]:
# Classification report
classification = classification_report(y_test, xgb_pred)
print("Modelo 06 - Gradient boosting")
print()
print(classification)

In [ ]:
# Métricas modelo
precision = precision_score(y_test, model_gradient_boosting_pred)
Recall = recall_score(y_test, model_gradient_boosting_pred)
Accuracy = accuracy_score(y_test, model_gradient_boosting_pred)
F1_Score = f1_score(y_test, model_gradient_boosting_pred)

precisao = pd.DataFrame({
    
    "Metricas" : ["precision",
                 "Recall", 
                  "Accuracy", 
                  "F1_Score"],
    
    "Resultado": [precision,
                Recall, 
                Accuracy, 
                F1_Score]})

precisao.sort_values(by = "Resultado", ascending = False)

## Modelo 07 - Light Gradient Boosting Machine (LightGBM)

In [ ]:
%%time

# Importando biblioteca
import lightgbm as lgb

# Defina os hiperparâmetros do modelo
parametros = {
    'objective': 'binary',  # Para problemas de classificação binária
    'metric': 'binary_logloss',  # Métrica de avaliação
    'boosting_type': 'gbdt',  # Tipo de boosting
    'num_leaves': 31,  # Número máximo de folhas em cada árvore
    'learning_rate': 0.05,  # Taxa de aprendizado
}

# Crie um dataset LightGBM a partir dos dados
train_data = lgb.Dataset(X_train, label=y_train)

# Treine o modelo
num_round = 250  # Número de iterações
modelo_LightGBM = lgb.train(parametros, train_data, num_round)
modelo_LightGBM

In [ ]:
# Salvar o modelo em um arquivo
modelo_LightGBM.save_model('modelo_lightgbm.txt')

In [ ]:
# Faça previsões no conjunto de teste
modelo_pred = modelo_LightGBM.predict(X_test, num_iteration=modelo_LightGBM.best_iteration)

In [ ]:
# Converta as previsões em classes (0 ou 1)
y_pred_binario = [1 if pred > 0.5 else 0 for pred in modelo_pred]

In [ ]:
# Accuracy score modelo
accuracy_LightGBM_boosting = accuracy_score(y_test, y_pred_binario)
print("Acurácia - LightGBM: %.2f" % (accuracy_LightGBM_boosting * 100))

In [ ]:
# Confusion matrix
matrix_confusion7 = confusion_matrix(y_test, y_pred_binario)

print('Confusion matrix - LightGBM \n\n', matrix_confusion7)
print('\nTrue Positives(TP) = ', matrix_confusion7[0,0])
print('\nTrue Negatives(TN) = ', matrix_confusion7[1,1])
print('\nFalse Positives(FP) = ', matrix_confusion7[0,1])
print('\nFalse Negatives(FN) = ', matrix_confusion7[1,0])

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 5))

ax = plt.subplot()
sns.heatmap(matrix_confusion7, annot=True, ax = ax, fmt = ".1f", cmap="Paired"); 
ax.set_title('Confusion Matrix - LightGBM'); 
ax.xaxis.set_ticklabels(["Acesso", "Não acesso"]); 
ax.yaxis.set_ticklabels(["Acesso", "Não acesso"]);

In [ ]:
# Calcule a Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_binario)

# Calcule a AUC (Área sob a Curva ROC)
auc = roc_auc_score(y_test, y_pred_binario)

# Plote a Curva ROC
plt.figure(figsize=(20, 5))
plt.plot(fpr, tpr, label=f'Curva ROC LightGBM (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC LightGBM')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Classification report
classification = classification_report(y_test, y_pred_binario)
print("Modelo - 07 - LightGBM")
print("\n")
print(classification)

In [ ]:
# Métricas
precision = precision_score(y_test, y_pred_binario)
Recall = recall_score(y_test, y_pred_binario)
Accuracy = accuracy_score(y_test, y_pred_binario)
F1_Score = f1_score(y_test, y_pred_binario)

precisao = pd.DataFrame({
    
    "Metricas" : ["precision",
                 "Recall", 
                  "Accuracy", 
                  "F1_Score"],
    
    "Resultado": [precision,
                Recall, 
                Accuracy, 
                F1_Score]})

precisao.sort_values(by = "Resultado", ascending = False)

## Resultados - Modelos machine learning

In [ ]:
# Resultados modelo

modelos = pd.DataFrame({
    
    "Models" :["Regressão logistica",
               "Random Forest", 
               "Decision Tree",
               "Naive Bayes",
               "XGBoost",
               "Gradient boosting",
               "LightGBM"],

    "Acurácia" :[accuracy_regressao_logistica,
                 accuracy_random_forest, 
                 acuracia_decision_tree,
                 accuracy,
                 accuracy_XGBoost,
                 accuracy_model_gradient_boosting,
                 accuracy_LightGBM_boosting]})

modelos.sort_values(by = "Acurácia", ascending = False)

# Conclusão

Este projeto de machine learning teve como foco a classificação de diferentes modelos, incluindo XGBoost, Gradient Boosting e Decision Tree. A principal pergunta abordada foi: "Um Lead será convertido? Sim ou Não? Qual a probabilidade?". Durante a análise, foram exploradas várias técnicas de modelagem preditiva para responder a essa questão.

Na primeira parte do projeto, exploramos a aplicação de diferentes algoritmos de classificação, como XGBoost, Gradient Boosting e Decision Tree, para determinar a probabilidade de conversão de leads. Esses modelos foram treinados e avaliados para identificar a abordagem mais eficaz na previsão de conversões. 

Na terceira parte do projeto, foi a fase de previsão, na qual nosso objetivo era responder à pergunta-chave: "O lead será convertido?". O resultado obtido indicou que a resposta do modelo "sim", com uma probabilidade de compra de 65%. 

Esse resultado é altamente relevante para estratégias de marketing digital, pois fornece informações valiosas para direcionar recursos e esforços de marketing de forma mais eficiente.

Este projeto destaca a importância da aplicação de machine learning em marketing digital. Ele demonstra como modelos de classificação podem ser aproveitados para prever a probabilidade de conversão de leads, fornecendo insights cruciais para otimizar campanhas e maximizar o retorno sobre o investimento. 

Em resumo, trata-se de um projeto fascinante que ilustra a relevância e o potencial da inteligência artificial no contexto do marketing digital.

# Referência

Link 1 - https://www.datageeks.com.br/xgboost/
    
Link 2 - https://lamfo.unb.br/wp-content/uploads/2020/08/Oficina-Random-Forest-e-LGMB.pdf
    
Link 3 - https://acervolima.com/lightgbm-vs-xgboost-qual-algoritmo-e-melhor/

# Citação

**Esse projeto da formação cientista de dados da @Data Science Academy**